In [ ]:
#| default_exp label_advanced

In [ ]:
#| export 
import pandas as pd
import openai
import os
import matplotlib.pyplot as plt

In [ ]:
#| export
def classify_unlabeled_texts(df, text_column, label_column):
    # Filter to get labeled examples for the prompt
    labeled_examples = df[df[label_column].notna()]
    openai.api_key = os.getenv('OPENAI_API_KEY')

    # Iterate over rows that need classification
    for index, row in df[df[label_column].isna()].iterrows():
        messages = [{"role": "system", "content": "The following are examples of texts and their classifications, the label is either 'OTR', 'PRS', 'REP', or 'NEU'. 'OTR' stands for 'An opportunity to respond' (e.g., 'What color is this candy?'), 'PRS' stands for 'Praise' (e.g., 'Great job, Morgan!'), 'REP' stands for 'Reprimand' (e.g., 'You need to keep quiet while someone else is reading.) and 'NEU' stands for 'None of the above':"}]
        
        # Add few-shot learning examples
        for _, example_row in labeled_examples.iterrows():
            example = f"Text: {example_row[text_column]}\nLabel: {example_row[label_column]}"
            messages.append({"role": "user", "content": example})

        # Add the text to be classified
        text_to_classify = row[text_column]
        messages.append({"role": "user", "content": f"Text: {text_to_classify}\nLabel:"})

        # Make the request to OpenAI's chat API
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        # Update the DataFrame with the classified label
        classified_label = response.choices[0].message['content'].strip()
        df.at[index, label_column] = classified_label

    return df

In [ ]:
#| export
def start_to_label(uploaded_file, chunk_size, text_column="Text", label_column="Label"):
    df = pd.read_csv(uploaded_file.name, encoding='utf-8')
    df = df[[text_column, label_column]]
    return df.head(int(chunk_size)), df

In [ ]:
#| export
def correct_dataframe(dataframe_display, file_input, current_dataframe, accuracy_log, chunk_size, chunk_count, text_column = "Text", label_column="Label"):
    max_chunk_count = len(current_dataframe) / chunk_size if len(current_dataframe) % chunk_size == 0 else int(len(current_dataframe) / chunk_size) + 1
    
    
    temp_df = current_dataframe.copy()


    # only iterate through the rows that are not empty
    
    for _, row in dataframe_display[~dataframe_display[label_column].isna()].iterrows():
        # current_dataframe[current_dataframe[text_column] == row[text_column]][label_column] = row[label_column]

        # use .loc[row_indexer,col_indexer] = value instead
        current_dataframe.loc[current_dataframe[text_column] == row[text_column], label_column] = row[label_column]

    # Check if "_AI_assisted.csv" is in the name of file_output
    accuracy = None

    if chunk_count > 1:
        compared_chunk_temp = temp_df.loc[min(chunk_size * (chunk_count - 1), len(temp_df) - chunk_size): min(chunk_size * chunk_count - 1, len(temp_df) - 1)]
        compared_chunk_current = current_dataframe.loc[min(chunk_size * (chunk_count - 1), len(current_dataframe) - chunk_size): min(chunk_size * chunk_count - 1, len(current_dataframe) - 1)]
        # Calculate the accuracy based on how many rows of the dataframe_display is different from the current_dataframe
        merged_df = pd.merge(compared_chunk_temp, compared_chunk_current, on=text_column, suffixes=('_before', '_current'))
        merged_df.dropna(subset=[f'{label_column}_current', f'{label_column}_before'], inplace=True)

        # with open('merged_df.csv', 'w') as f:
        #     f.write(merged_df.to_csv(merged_df[f'{label_column}_before'] != merged_df[f'{label_column}_current']).sum())
        accuracy = 1 - (merged_df[f'{label_column}_before'] != merged_df[f'{label_column}_current']).sum() / len(merged_df)
        if len(accuracy_log) < max_chunk_count - 1:
            accuracy_log.append(accuracy)

    if accuracy != 1:
        current_dataframe.loc[min(chunk_size * chunk_count, len(current_dataframe) - chunk_size): , label_column] = None
        current_dataframe = classify_unlabeled_texts(current_dataframe, 'Text', 'Label')

    new_file = file_input.name.split('.')[0] + '_AI_assisted.csv'

    current_dataframe.to_csv(new_file)

    dataframe_display = current_dataframe.loc[min(chunk_size * chunk_count, len(current_dataframe) - chunk_size): min(chunk_size*(chunk_count+1)-1, len(current_dataframe) - 1)]

    if chunk_count < max_chunk_count - 1:
        chunk_count += 1

    return dataframe_display, new_file, current_dataframe, accuracy_log, chunk_count

In [ ]:
#| export
def create_plot(accuracy_log, save_path="plot.png"):
    ax = plt.figure()
    # X is the index of the accuracy_log, y is the value in accuray_log
    plt.plot(range(len(accuracy_log)), accuracy_log)
    plt.xlabel('Batch Index')
    plt.ylabel('Batch Accuracy')
    plt.title('Batch Accuracy over time')

    # Save the plot to a BytesIO object
    # buf = io.BytesIO()
    plt.savefig(save_path, format='png')
    # buf.seek(0)

    # Return the plot and the file for download
    return ax, save_path

In [ ]:
# key_dict = json.load(open('../api_key.json', 'r'))
# for key in key_dict:
#     os.environ[key] = key_dict[key]

# df = pd.read_csv('../data/009-1.csv')
# response = classify_unlabeled_texts(df, 'Text', 'Label')
# print(response)
# print(new_df)

In [ ]:
# print(response['Label'].isna().sum())